In [1]:
import pandas as pd
import os
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
src_folder = os.path.join(project_root, 'src')
sys.path.insert(0, src_folder)

from pre_proc import *

# The problem domain

The FAB (Força Aérea Brasileira) makes available a dataset containing the occurrences of the civil aviation in Brazil from 2008 to 2017. The [figure](#data_model) shows the data model for the data.

<a id='data_model_id'></a> <img src="../resources/data_model.png">

For each ocurrence, there is a classification for it: incident, serious incident, and accident. __The objective is to develop a statistical model to classify each ocurrence in its THREE categories.__

## The data

Let's analyse the first rows of each table to have an idea about what each column stores.

In [2]:
df_occurrences = pd.read_csv("../data/raw/occurrences.csv", sep = '~')
df_aircrafts = pd.read_csv("../data/raw/aircrafts.csv", sep = '~')
df_factors = pd.read_csv("../data/raw/factors.csv", sep = '~')
df_recommendations = pd.read_csv("../data/raw/recommendations.csv", sep = '~')

In [3]:
df_occurrences.head()

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_tipo,ocorrencia_tipo_categoria,ocorrencia_tipo_icao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,ocorrencia_pais,...,ocorrencia_horario,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista,ocorrencia_dia_extracao
0,201305055424986,ACIDENTE,FALHA DO MOTOR EM VOO,FALHA OU MAU FUNCIONAMENTO DO MOTOR,SCF-PP,-9.9085,-63.0333,ARIQUEMES,RO,BRASIL,...,11:00:00,NaN,FINALIZADA,***,NÃO,NaN,0,1,NÃO,2019-05-31
1,201311259977425,INCIDENTE GRAVE,POUSO SEM TREM,CONTATO ANORMAL COM A PISTA,ARC,-11.4956,-61.4508,CACOAL,RO,BRASIL,...,12:32:00,NaN,FINALIZADA,IG-209/CENIPA/2013,SIM,2014-04-07,0,1,NÃO,2019-05-31
2,201605160250139,INCIDENTE GRAVE,COLISÃO COM OBSTÁCULO DURANTE A DECOLAGEM E POUSO,COLISÃO COM OBSTÁCULO DURANTE A DECOLAGEM E POUSO,CTOL,-11.2644,-61.2589,CACOAL,RO,BRASIL,...,19:19:00,SIM,FINALIZADA,IG-084/CENIPA/2016,SIM,2019-03-11,7,1,NÃO,2019-05-31
3,201805021421302,INCIDENTE,AERÓDROMO,AERÓDROMO,ADRM,-11.4956,-61.4508,CACOAL,RO,BRASIL,...,22:40:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO,2019-05-31
4,201103187273112,INCIDENTE,OUTROS,OUTROS,OTHR,-13.1950,-60.8184,CEREJEIRAS,RO,BRASIL,...,13:30:00,NaN,FINALIZADA,***,NÃO,NaN,0,1,NÃO,2019-05-31


In [4]:
df_aircrafts.head()

,codigo_ocorrencia,aeronave_matricula,aeronave_operador_categoria,aeronave_tipo_veiculo,aeronave_fabricante,aeronave_modelo,aeronave_tipo_icao,aeronave_motor_tipo,aeronave_motor_quantidade,aeronave_pmd,...,aeronave_registro_categoria,aeronave_registro_segmento,aeronave_voo_origem,aeronave_voo_destino,aeronave_fase_operacao,aeronave_fase_operacao_icao,aeronave_tipo_operacao,aeronave_nivel_dano,total_fatalidades,aeronave_dia_extracao
0,201106142171203,PPGXE,AEROCLUBE,AVIÃO,NEIVA INDUSTRIA AERONAUTICA,56-C,PAUL,PISTÃO,MONOMOTOR,660,...,PRI,INSTRUÇÃO,SDPW,SDPW,INDETERMINADA,UNKNOWN,VOO DE INSTRUÇÃO,SUBSTANCIAL,0,2019-05-31
1,201205209591320,PTRBN,OPERADOR DE AERONAVE,AVIÃO,EMBRAER,EMB-810C,PA34,PISTÃO,BIMOTOR,2073,...,TPX,TÁXI AÉREO,****,****,APROXIMAÇÃO FINAL,APPROACH,TÁXI AÉREO,NENHUM,0,2019-05-31
2,201012015549851,PTKUK,OPERADOR DE AERONAVE,AVIÃO,TWIN COMMANDER,500-B,AC50,PISTÃO,BIMOTOR,3062,...,TPX,TÁXI AÉREO,****,****,CORRIDA APÓS POUSO,LANDING,TÁXI AÉREO,LEVE,0,2019-05-31
3,201708190325167,PTKUK,OPERADOR PARTICULAR,AVIÃO,TWIN COMMANDER,500-B,AC50,PISTÃO,BIMOTOR,3062,...,SAE,ESPECIALIZADA,SBCP,SBCP,CRUZEIRO,EN ROUTE,OPERAÇÃO ESPECIALIZADA,LEVE,0,2019-05-31
4,201803182255192,PPGSZ,AEROCLUBE,AVIÃO,NEIVA INDUSTRIA AERONAUTICA,56-C,PAUL,PISTÃO,MONOMOTOR,660,...,PRI,INSTRUÇÃO,SBBP,SBBP,ARREMETIDA NO SOLO,APPROACH,VOO DE INSTRUÇÃO,LEVE,0,2019-05-31


In [5]:
df_factors.head()

,codigo_ocorrencia,fator_nome,fator_aspecto,fator_condicionante,fator_area,fator_detalhe_fator,fator_dia_extracao
0,200901015424167,JULGAMENTO DE PILOTAGEM,DESEMPENHO DO SER HUMANO,OPERAÇÃO DA AERONAVE,FATOR OPERACIONAL,"A PROVÁVEL INTENÇÃO EM PRESERVAR A AERONAVE, C...",2019-05-31
1,200901015424167,MANUTENÇÃO DE AERONAVE,DESEMPENHO DO SER HUMANO,MANUTENÇÃO DA AERONAVE,FATOR OPERACIONAL,O RELATÓRIO TÉCNICO DO IAE REVELOU FALHAS ORIU...,2019-05-31
2,200901015424167,SUPERVISÃO GERENCIAL,DESEMPENHO DO SER HUMANO,OPERAÇÃO DA AERONAVE,FATOR OPERACIONAL,O OPERADOR NÃO PROGRAMOU A REALIZAÇÃO DE INSPE...,2019-05-31
3,200901055963381,ATITUDE,ASPECTO PSICOLÓGICO,INDIVIDUAL,FATOR HUMANO,O PILOTO ERA POUCO EXPERIENTE E REALIZOU O POU...,2019-05-31
4,200901055963381,PROCESSO DECISÓRIO,ASPECTO PSICOLÓGICO,INDIVIDUAL,FATOR HUMANO,"A POUCA EXPERIÊNCIA DO PILOTO, ALIADA AO EXCES...",2019-05-31


In [6]:
df_recommendations.head()

,codigo_ocorrencia,recomendacao_numero,recomendacao_dia_assinatura,recomendacao_dia_encaminhamento,recomendacao_feedback,recomendacao_conteudo,recomendacao_status,recomendacao_destinatario_sigla,recomendacao_destinatario_nome,dia_extracao_recomendacao
0,200901015424167,123/2012,2012-03-22,2012-04-20,NaN,Dar ampla divulgação aos operadores de aeronav...,AGUARDANDO RESPOSTA,ABAAC,ASSOCIAÇÃO BRASILEIRA DE AERONAVES ANTIGAS E C...,2019-05-31
1,200901015424167,122/2012,2012-03-22,2012-04-20,2012-11-26,Realizar fiscalização mais rigorosa na vistori...,CUMPRIDA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL,2019-05-31
2,200901015424167,121/2012,2012-03-22,2012-04-20,NaN,Aplicar processos que melhorem a qualidade da ...,AGUARDANDO RESPOSTA,AEROCAMP,AEROCAMP COMÉRCIO E SERVIÇOS AERONÁUTICOS,2019-05-31
3,200901015424167,120/2012,2012-03-22,2012-04-20,NaN,Promover a atualização de todos os registros d...,AGUARDANDO RESPOSTA,EXTREME,EXTREME DEMONSTRAÇÕES AÉREAS,2019-05-31
4,200901015424167,119/2012,2012-03-22,2012-04-20,NaN,Enfatizar o estrito cumprimento dos procedimen...,AGUARDANDO RESPOSTA,EXTREME,EXTREME DEMONSTRAÇÕES AÉREAS,2019-05-31


Table _occurrences_ shows informations such as category/type, local, and time of the occurrence. Table _aircrafts_ presents the characteristics of the aircraft involved on the occurence, for example, type and number of engines. Table _factors_ depicts informations about reasons for causing the occurrence, which one can extracts human or mechanical factors, for instance. Finally, the last table _recommendations_ shows councelings to avoid this occurrence in the future, thus it is have no information to train a model for prediction.

Therefore, based on the information above, for the next steps __we can discard the table _recommendations_.__

In [7]:
cur_dir = os.getcwd()

src_dir = os.path.join(cur_dir, os.pardir, "data/raw")
dst_dir = os.path.join(cur_dir, os.pardir, "data/interm_intro")
files = ["occurrences.csv", "aircrafts.csv", "factors.csv"]

copy_files(src_dir, dst_dir, files)